<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/6%20-%20Generatori%20e%20Images%20Augmentation/images_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Images Augmentation
Per addestrare una rete neurale, specialmente una molto profonda, è necessario avere a disposizione una grande quantità di dati. Se il dataset che abbiamo è limitato, una soluzione potrebbe consistere nel generare nuovi dati sintetici partendo da quelli di cui già disponiamo. Nell'ambito della computer vision questa pratica è chiamata Images Aumentation, in questo notebook vedremo come applicarla utilizzando i generatori di tf.keras.

## Scarichiamo il Dataset
Anche in questo notebook proveremo a classificare immagini di dessert, ma questa volta avremo a disposizione soltanto 1000 immagini in totale per l'addestramento, 200 per ogni classe.

In [5]:
!wget https://profession.ai/datasets/dessert_small.zip
!unzip -qq dessert_small.zip

--2020-06-29 11:12:10--  https://profession.ai/datasets/dessert_small.zip
Resolving profession.ai (profession.ai)... 13.224.166.2, 13.224.166.63, 13.224.166.122, ...
Connecting to profession.ai (profession.ai)|13.224.166.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52003499 (50M) [application/zip]
Saving to: ‘dessert_small.zip’

dessert_small.zip   100%[===================>]  49.59M  15.3MB/s    in 4.1s    

2020-06-29 11:12:15 (12.1 MB/s) - ‘dessert_small.zip’ saved [52003499/52003499]



## Importiamo i Moduli

In [6]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Definiamo le Costanti

In [7]:
DATASET_DIR = "dessert_small/"
BATCH_SIZE = 64

TOT_IMAGES = 1000 
IMG_SIZE = (228, 228)

## Usiamo il Generatori per Generare nuove Immagini
Per generare nuove immagini dobbiamo applicare delle trasformazioni alle immagini di cui già disponiamo, come rotazioni, translazioni o zoom. Per definire diverse trasformazioni da applicare possiamo utilizzare i parametri della classe *ImageDataGenerator*. [Qui](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) puoi trovare la lista di tutti i parametri e di tutte le possibili trasformazioni.

In [8]:
datagen_train = ImageDataGenerator(validation_split=0.2, 
                            rescale=1./255,
                            shear_range=0.2, 
                            zoom_range=0.2, 
                            horizontal_flip=True)

Adesso possiamo creare i generatori di immagini per addestramento e validazione.

In [9]:
train_generator = datagen_train.flow_from_directory(
        DATASET_DIR+"train", 
        target_size=IMG_SIZE, 
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training')

valid_generator = datagen_train.flow_from_directory(
        DATASET_DIR+"train",
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation')


print(train_generator.class_indices)

Found 800 images belonging to 5 classes.
Found 200 images belonging to 5 classes.
Found 1250 images belonging to 5 classes.
{'cannoli': 0, 'gelato': 1, 'panna_cotta': 2, 'tiramisu': 3, 'torta_di_mele': 4}


Creiamo il generatore per il test, in questo caso non dobbiamo applicare trasformazioni.

In [ ]:
datagen_test = ImageDataGenerator(validation_split=0.2, rescale=1./255)


test_generator = datagen_test.flow_from_directory(
        DATASET_DIR+"test",
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

## Addestriamo la Rete Neurale Convoluzionale sul Generatore
Utilizziamo la stessa architettura che abbiamo definito per l'esempio dei generatori.

In [26]:
#regularizer = tf.keras.regularizers.L1L2(l1=0.01, l2=0.1)
regularizer = tf.keras.regularizers.L1L2(l2=0.1)

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dropout(0.5))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(5, activation='softmax'))

print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 228, 228, 64)      1792      
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 228, 228, 64)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 76, 76, 64)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 76, 76, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 76, 76, 32)        18464     
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 76, 76, 32)        9248      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 25, 25, 32)       

Il processo di addestramento è esattamente ugualea quanto già fatto per i generatori.

In [27]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                              min_delta=0.01,
                              patience=3,
                              restore_best_weights=True)

model.fit(
        train_generator,
        steps_per_epoch=int(TOT_IMAGES*0.8 // BATCH_SIZE),
        validation_data=valid_generator, 
        validation_steps=int(TOT_IMAGES*0.2 // BATCH_SIZE),
        epochs=100)

Epoch 1/100
12/12 [==============================] - 16s 1s/step - loss: 1.6895 - accuracy: 0.1848 - val_loss: 1.6094 - val_accuracy: 0.2083
Epoch 2/100
12/12 [==============================] - 15s 1s/step - loss: 1.6100 - accuracy: 0.2038 - val_loss: 1.6094 - val_accuracy: 0.1927
Epoch 3/100
12/12 [==============================] - 15s 1s/step - loss: 1.6093 - accuracy: 0.1984 - val_loss: 1.6094 - val_accuracy: 0.1979
Epoch 4/100
12/12 [==============================] - 16s 1s/step - loss: 1.6094 - accuracy: 0.1970 - val_loss: 1.6094 - val_accuracy: 0.1927
Epoch 5/100
12/12 [==============================] - 16s 1s/step - loss: 1.6074 - accuracy: 0.2092 - val_loss: 1.6071 - val_accuracy: 0.2031
Epoch 6/100
12/12 [==============================] - 16s 1s/step - loss: 1.6026 - accuracy: 0.2310 - val_loss: 1.6091 - val_accuracy: 0.2344
Epoch 7/100
12/12 [==============================] - 16s 1s/step - loss: 1.5989 - accuracy: 0.2331 - val_loss: 1.6057 - val_accuracy: 0.2083
Epoch 8/100
1

# Testiamo la Rete sul Generatore
Per testare sul generatore possiamo sempre usare il metodo *evaluate*.

In [28]:
metrics_train = model.evaluate(train_generator)
metrics_test = model.evaluate(test_generator)

print("Train Accuracy = %.4f - Train Loss = %.4f" % (metrics_train[1], metrics_train[0]))
print("Test Accuracy = %.4f - Test Loss = %.4f" % (metrics_test[1], metrics_test[0]))

20/20 [==============================] - 18s 911ms/step - loss: 2.2509 - accuracy: 0.4144
Train Accuracy = 0.9725 - Train Loss = 0.1707
Test Accuracy = 0.4144 - Test Loss = 2.2509
